In [1]:
import sqlalchemy
sqlalchemy.__version__

'1.3.11'

In [2]:
from sqlalchemy import create_engine

In [3]:
engine = create_engine('sqlite:///:memory:', echo=True) # echo 标志是日志记录的快捷方式，可以看到生成的 sql
# create_engine 创建了一个 engine 示例，但并不会立即连接数据库

In [4]:
from sqlalchemy.ext.declarative import declarative_base
#创建声明式系统映射类的基类,一般程序中仅具有该基类的一个实例
Base = declarative_base()

In [5]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'user'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)
    
    def __repr__(self):
        return f'<User(name={self.name}, fullname={self.fullname}, nickname={self.nickname})>'
    

In [6]:
Base.metadata.create_all(engine)

2019-11-28 18:38:11,541 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-28 18:38:11,542 INFO sqlalchemy.engine.base.Engine ()
2019-11-28 18:38:11,543 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-28 18:38:11,543 INFO sqlalchemy.engine.base.Engine ()
2019-11-28 18:38:11,544 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("user")
2019-11-28 18:38:11,546 INFO sqlalchemy.engine.base.Engine ()
2019-11-28 18:38:11,547 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("user")
2019-11-28 18:38:11,547 INFO sqlalchemy.engine.base.Engine ()
2019-11-28 18:38:11,548 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2019-11-28 18:38:11,549 INFO sqlalchemy.engine.base.Engine ()
2019-11-28 18:38:11,549 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
ed_user = User(name='ed', fullname='Ed jones', nickname='edsnickname')

In [8]:
ed_user.name

'ed'

In [9]:
ed_user.nickname

'edsnickname'

In [10]:
str(ed_user.id)

'None'

In [11]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)

In [12]:
session = Session()

In [13]:
session.add(ed_user)

In [14]:
our_user = session.query(User).filter_by(name='ed').first()

2019-11-28 18:38:11,606 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-28 18:38:11,608 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-28 18:38:11,608 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed jones', 'edsnickname')
2019-11-28 18:38:11,610 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?
2019-11-28 18:38:11,611 INFO sqlalchemy.engine.base.Engine ('ed', 1, 0)


In [15]:
ed_user is our_user

True

In [16]:
session.add_all([
    User(name='wendy', fullname='Wendy Williams', nickname='windy'),
    User(name='mary', fullname='Mary Contrary', nickname='mary'),
    User(name='fred', fullname='Fred Flintstone', nickname='freddy')
])

In [17]:
ed_user.nickname='eddie'

In [18]:
session.dirty

IdentitySet([<User(name=ed, fullname=Ed jones, nickname=eddie)>])

In [19]:
session.new

IdentitySet([<User(name=wendy, fullname=Wendy Williams, nickname=windy)>, <User(name=mary, fullname=Mary Contrary, nickname=mary)>, <User(name=fred, fullname=Fred Flintstone, nickname=freddy)>])

In [20]:
session.commit()

2019-11-28 18:38:11,648 INFO sqlalchemy.engine.base.Engine UPDATE user SET nickname=? WHERE user.id = ?
2019-11-28 18:38:11,649 INFO sqlalchemy.engine.base.Engine ('eddie', 1)
2019-11-28 18:38:11,649 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-28 18:38:11,653 INFO sqlalchemy.engine.base.Engine ('wendy', 'Wendy Williams', 'windy')
2019-11-28 18:38:11,655 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-28 18:38:11,656 INFO sqlalchemy.engine.base.Engine ('mary', 'Mary Contrary', 'mary')
2019-11-28 18:38:11,656 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-28 18:38:11,658 INFO sqlalchemy.engine.base.Engine ('fred', 'Fred Flintstone', 'freddy')
2019-11-28 18:38:11,658 INFO sqlalchemy.engine.base.Engine COMMIT


In [21]:
ed_user.id

2019-11-28 18:38:11,662 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-28 18:38:11,663 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user 
WHERE user.id = ?
2019-11-28 18:38:11,663 INFO sqlalchemy.engine.base.Engine (1,)


1

In [22]:
ed_user.name = 'Edwardo'

In [23]:
fake_user = User(name='fakeuser', fullname='Invalid', nickname='123456')
session.add(fake_user)

In [24]:
session.query(User).filter(User.name.in_(['Edwardo', 'fakeuser'])).all()

2019-11-28 18:38:11,686 INFO sqlalchemy.engine.base.Engine UPDATE user SET name=? WHERE user.id = ?
2019-11-28 18:38:11,689 INFO sqlalchemy.engine.base.Engine ('Edwardo', 1)
2019-11-28 18:38:11,690 INFO sqlalchemy.engine.base.Engine INSERT INTO user (name, fullname, nickname) VALUES (?, ?, ?)
2019-11-28 18:38:11,692 INFO sqlalchemy.engine.base.Engine ('fakeuser', 'Invalid', '123456')
2019-11-28 18:38:11,694 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user 
WHERE user.name IN (?, ?)
2019-11-28 18:38:11,695 INFO sqlalchemy.engine.base.Engine ('Edwardo', 'fakeuser')


[<User(name=Edwardo, fullname=Ed jones, nickname=eddie)>,
 <User(name=fakeuser, fullname=Invalid, nickname=123456)>]

In [25]:
fake_user in session

True

In [26]:
session.rollback()

2019-11-28 18:38:11,708 INFO sqlalchemy.engine.base.Engine ROLLBACK


In [27]:
ed_user.name

2019-11-28 18:38:11,716 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-28 18:38:11,718 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user 
WHERE user.id = ?
2019-11-28 18:38:11,719 INFO sqlalchemy.engine.base.Engine (1,)


'ed'

In [28]:
fake_user in session

False

In [29]:
session.query(User).filter(User.name.in_(['ed', 'fakeuser'])).all()

2019-11-28 18:38:11,730 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user 
WHERE user.name IN (?, ?)
2019-11-28 18:38:11,731 INFO sqlalchemy.engine.base.Engine ('ed', 'fakeuser')


[<User(name=ed, fullname=Ed jones, nickname=eddie)>]

In [30]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

2019-11-28 18:38:11,739 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user ORDER BY user.id
2019-11-28 18:38:11,740 INFO sqlalchemy.engine.base.Engine ()
ed Ed jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [31]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2019-11-28 18:38:11,745 INFO sqlalchemy.engine.base.Engine SELECT user.name AS user_name, user.fullname AS user_fullname 
FROM user
2019-11-28 18:38:11,746 INFO sqlalchemy.engine.base.Engine ()
ed Ed jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [32]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2019-11-28 18:38:11,752 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name, user.fullname AS user_fullname, user.nickname AS user_nickname 
FROM user
2019-11-28 18:38:11,753 INFO sqlalchemy.engine.base.Engine ()
<User(name=ed, fullname=Ed jones, nickname=eddie)> ed
<User(name=wendy, fullname=Wendy Williams, nickname=windy)> wendy
<User(name=mary, fullname=Mary Contrary, nickname=mary)> mary
<User(name=fred, fullname=Fred Flintstone, nickname=freddy)> fred


/home/kevin/.virtualenvs/StudyPython-AS8qU0JI/lib/python3.6/site-packages/sqlalchemy/ext/declarative/clsregistry.py:129: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Address, and will be replaced in the string-lookup table.
  % (item.__module__, item.__name__)


InvalidRequestError: Table 'address' is already defined for this MetaData instance.  Specify 'extend_existing=True' to redefine options and columns on an existing Table object.

In [35]:
User.addresses= relationship('Address', order_by=Address.id, back_populates='user')

NoForeignKeysError: Could not determine join condition between parent/child tables on relationship User.address - there are no foreign keys linking these tables.  Ensure that referencing columns are associated with a ForeignKey or ForeignKeyConstraint, or specify a 'primaryjoin' expression.